In [1]:
def extraer_informacion(url, archivo_csv):
    import requests
    from bs4 import BeautifulSoup
    import urllib.parse
    import csv

    # Crea una lista para almacenar los enlaces
    link_list = []

    # Establece un límite para el número de páginas a procesar
    limit = 500

    # Establece el offset inicial
    of = 0

    while of<limit:
        # Realiza una solicitud HTTP a la página web con el offset actual
        response = requests.get(f"{url}/recent-submissions?offset={of}")

        # Obtiene el HTML de la respuesta
        html = response.text

        # Crea un objeto BeautifulSoup a partir del HTML
        soup = BeautifulSoup(html, "html.parser")
        
        # Parsea la URL
        parsed_url = urllib.parse.urlparse("{}/recent-submissions?offset={}".format(url, of))
        # Obtiene el contenido principal de la URL (todo antes del /repositorio)
        url_base = parsed_url.scheme + '://' + parsed_url.netloc

        # Busca todos los enlaces <a> en el HTML con clase image-link 
        # Esto permite buscar los hrefs que estan ligados a las imagenes(thumbnails) de cada tesis
        # Solo funciona si el repositorio usa thumbnails
        links = soup.find_all("a",{"class":"image-link"})
        hrefs1 = [url_base + a["href"] for a in links]

        # Itera sobre cada enlace y agrega su atributo "href" a la lista
        for link in hrefs1:
            link_list.append(link)

        # Incrementa el offset para la siguiente iteración
        of += 20

    # Acesso a la tabla de metadatos añadiendo ?show=full
    links_full = [link + '?show=full' for link in link_list]

    # Abre un archivo CSV en modo escritura
    with open(archivo_csv, 'w', newline='') as csvfile:
        # Crea el objeto escritor
        writer = csv.writer(csvfile)
        # Escribe la cabecera del archivo CSV
        writer.writerow(['Autores', 'Asesores', 'Título', 'Resumen', 'Año', 'Institución', 'Grado'])

        # Recorre cada link
        for link in links_full:
            # Descarga el contenido del link
            html = requests.get(link).text
            # Parsea el contenido del link
            soup = BeautifulSoup(html, 'html.parser')

            # Obtiene la información de cada link
            authors = [element['content'] for element in soup.find_all('meta', {'name': 'citation_author'})]
            asesores = [element.find_next_sibling().text for element in soup.find_all('td', string='dc.contributor.advisor')]
            titulo = soup.find('meta', {'name': 'citation_title'})['content']
            abstract = soup.find('meta', {'name': "DCTERMS.abstract"})['content']
            año = soup.find('meta', {'name': "citation_date"})['content']
            institucion = soup.find('meta', {'name': 'citation_publisher'})['content']
            grado = [element.find_next_sibling().text for element in soup.find_all('td', string='thesis.degree.name')]

            # Escribe la fila en el archivo CSV
            writer.writerow([authors, asesores, titulo, abstract, año, institucion, grado])

,Autores,Asesores,Título,Resumen,Año,Institución,Grado
0,"['Calderon Torrejon, Flavio', 'Boza Quilca, Ro...","['Minauro Latorre, Tomás Alberto']",Plan de negocio para determinar la viabilidad ...,Implementación de un sistema de monitoreo de l...,2022,Universidad ESAN,['Magíster en Dirección de Tecnologías de la I...
1,"['Accho Rojas, Vladimir Braulio', 'Alejandria ...","['Moarri Nohra, Richard']",Modelo de implementación del proceso de emisió...,En los últimos años se ha incrementado el núme...,2022,Universidad ESAN,['Magíster en Dirección de Tecnologías de la I...
2,"['Tang Arce, Fernando Andres', 'Agapito Almeyd...","['Serida Nishimura, Jaime']",Plan de negocio para la implementación de una ...,En el Perú no existe una solución digital dond...,2022,Universidad ESAN,['Magíster en Dirección de Tecnologías de la I...
3,"['Bolaños de la Cruz, Maria Petronila', 'Manri...","['Moarri Nohra, Richard']",Modelo y plan de negocio para la implementació...,En ciertas ocasiones nos ha pasado que dejamos...,2022,Universidad ESAN,['Magíster en Dirección de Tecnologías de la I...
4,"['Toledo Sánchez, Luzmila Zenaida', 'Pinto Val...","['Cuervo Guzmán, Sergio']",Plan de negocio para la implementación de una ...,"“Orient@-T”, es una plataforma web y móvil que...",2021,Universidad ESAN,['Magíster en Dirección de Tecnologías de la I...
...,...,...,...,...,...,...,...
84,"['Chávez Prada, Henry Humberto', 'Vega Asenció...","['Bresani Torres, Aldo']",Propuesta de modelo de continuidad de negocio ...,El objetivo principal de esta tesis es la prop...,2015,Universidad ESAN,['Maestro/Magíster en Dirección de Tecnologías...
85,"['Carmona Yataco, Claudia Cristina', 'Chinchay...","['Santana Ormeño, Martín']",Plan de negocio para ofrecer servicios móviles...,La presente tesis tiene como objetivo general ...,2016,Universidad ESAN,['Maestro/Magíster en Dirección de Tecnologías...
86,"['Bullón Vallejo, Alfonso José', 'Tacuche Cond...","['Santana Ormeño, Martín']",Plan de negocio para la creación de un negocio...,De un tiempo a esta parte el mercado de cupone...,2016,Universidad ESAN,['Maestro/Magíster en Dirección de Tecnologías...
87,"['Auris Rodríguez, Mario David', 'Tapia López,...","['Morris Abarca, Eddy', 'Salcedo Zuta, Néstor ...",Determinantes de la aceptación y usabilidad de...,El motivo de la presente investigación es iden...,2016,Universidad ESAN,['Maestro/Magíster en Dirección de Tecnologías...


In [53]:
df.iloc[:, 2]

0     Plan de negocio para determinar la viabilidad ...
1     Modelo de implementación del proceso de emisió...
2     Plan de negocio para la implementación de una ...
3     Modelo y plan de negocio para la implementació...
4     Plan de negocio para la implementación de una ...
                            ...                        
84    Propuesta de modelo de continuidad de negocio ...
85    Plan de negocio para ofrecer servicios móviles...
86    Plan de negocio para la creación de un negocio...
87    Determinantes de la aceptación y usabilidad de...
88    Plan de negocio : elaboración de una herramien...
Name: Título, Length: 89, dtype: object